In [50]:
import keras
from keras.models import Sequential
from keras.layers import Flatten, Conv2D, Dense, MaxPool2D
from keras import utils

from glob import glob
from matplotlib import pyplot as plt
import numpy as np
from PIL import Image

In [4]:
base_path = "../input/tryponet_set2.tar.gz/tryponet_set2/"

In [5]:
def load_Xy(path):
    filenames0 = glob(path + "norm/*.png")
    filenames1 = glob(path + "trypo/*.png")
    X = np.zeros((len(filenames0) + len(filenames1), 256, 256, 3))
    
    y = np.zeros(len(filenames0) + len(filenames1))
    y[len(filenames0):] = 1.
    
    print(path + "norm/*.png")
    for i, filename in enumerate(filenames0):
        X[i] = plt.imread(filename)
        if i % 100 == 0:
            print(i, end=" ")

    print("\n")
    print(path + "trypo/*.png")
    for i, filename in enumerate(filenames1):
        X[len(filenames0) + i] = plt.imread(filename)
        if i % 100 == 0:
            print(i, end=" ")
            
    return X, y

In [48]:
def array_2d_to_image(array, autorescale=True):
    assert array.min() >= 0
    assert len(array.shape) in [2, 3]
    if array.max() <= 1 and autorescale:
        array = 255 * array
    array = array.astype('uint8')
    return Image.fromarray(array)

def model_summary(model):
    print("Model created successfully.")
    print(model.summary())
    ctx.channel_send('n_layers', len(model.layers))
    ctx.channel_send('n_parameters', model.count_params())

categories = ['norm', 'trypo']

class NeptuneCallback(Callback):
    def __init__(self, x_test, y_test, images_per_epoch=-1):
        
        try:
            ctx.channel_reset('Log-loss training')
            ctx.channel_reset('Log-loss validation')
            ctx.channel_reset('Accuracy training')
            ctx.channel_reset('Accuracy validation')
            ctx.channel_reset('false_predictions')
        except:
            pass
        self.epoch_id = 0
        self.images_per_epoch = images_per_epoch
        self.x_test = x_test
        self.y_test = y_test

    def on_epoch_end(self, epoch, logs={}):
        self.epoch_id += 1

        # logging numeric channels
        ctx.channel_send('Log-loss training', self.epoch_id, logs['loss'])
        ctx.channel_send('Log-loss validation', self.epoch_id, logs['val_loss'])
        ctx.channel_send('Accuracy training', self.epoch_id, logs['acc'])
        ctx.channel_send('Accuracy validation', self.epoch_id, logs['val_acc'])

        # Predict the digits for images of the test set.
        validation_predictions = self.model.predict_classes(self.x_test)
        scores = self.model.predict(self.x_test)

        # Identify the incorrectly classified images and send them to Neptune Dashboard.
        image_per_epoch = 0
        for index, (prediction, actual) in enumerate(zip(validation_predictions, self.y_test.argmax(axis=1))):
            if prediction != actual:
                if image_per_epoch == self.images_per_epoch:
                    break
                image_per_epoch += 1

                ctx.channel_send('false_predictions', neptune.Image(
                    name='[{}] {} X {} V'.format(self.epoch_id, categories[prediction], categories[actual]),
                    description="\n".join([
                        "{:5.1f}% {} {}".format(100 * score, categories[i], "!!!" if i == actual else "")
                        for i, score in enumerate(scores[index])]),
                    data=array_2d_to_image(self.x_test[index,:,:])))

In [9]:
X_train, y_train = load_Xy(base_path + "train/")

../input/tryponet_set2.tar.gz/tryponet_set2/train/norm/*.png
0 100 200 300 400 500 600 700 800 900 1000 1100 1200 1300 1400 1500 1600 1700 1800 1900 2000 2100 2200 2300 2400 2500 2600 2700 2800 2900 3000 3100 3200 3300 3400 3500 3600 3700 3800 3900 4000 4100 4200 4300 4400 4500 4600 4700 4800 4900 5000 5100 5200 5300 5400 5500 5600 5700 5800 5900 6000 6100 6200 6300 6400 6500 6600 6700 6800 6900 7000 7100 7200 7300 7400 7500 7600 7700 7800 7900 8000 8100 8200 8300 8400 8500 8600 8700 8800 8900 9000 9100 9200 9300 9400 9500 9600 9700 9800 9900 10000 

../input/tryponet_set2.tar.gz/tryponet_set2/train/trypo/*.png
0 100 200 300 400 500 600 700 800 900 1000 1100 1200 1300 1400 1500 1600 1700 1800 1900 2000 2100 2200 2300 2400 2500 2600 2700 2800 2900 3000 3100 3200 3300 3400 3500 3600 3700 3800 3900 4000 4100 4200 4300 4400 4500 4600 4700 4800 4900 5000 5100 5200 5300 5400 5500 5600 5700 5800 

In [ ]:
X_test, y_test = load_Xy(base_path + "train/")

In [44]:
Y_train = utils.to_categorical(y_train, 2)
Y_test = utils.to_categorical(y_test, 2)

In [41]:
# network definition
model = Sequential()
model.add(Flatten(input_shape=(256, 256, 3)))
model.add(Dense(2, activation='softmax'))

In [42]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [43]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_2 (Flatten)          (None, 196608)            0         
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 393218    
Total params: 393,218
Trainable params: 393,218
Non-trainable params: 0
_________________________________________________________________


In [49]:
model.fit(X_train, Y_train,
          epochs=20,
          batch_size=32,
          validation_data=(X_test, Y_test),
          verbose=2,
          callbacks=[NeptuneCallback(X_test, Y_test, images_per_epoch=10)])

Train on 15884 samples, validate on 1000 samples
Epoch 1/5
 992/1000 [============================>.] - ETA: 0s23s - loss: 5.9018 - acc: 0.6338 - val_loss: 8.0590 - val_acc: 0.5000
Epoch 2/5
 992/1000 [============================>.] - ETA: 0s24s - loss: 5.9018 - acc: 0.6338 - val_loss: 8.0590 - val_acc: 0.5000
Epoch 3/5
 992/1000 [============================>.] - ETA: 0s24s - loss: 5.9018 - acc: 0.6338 - val_loss: 8.0590 - val_acc: 0.5000
Epoch 4/5
 992/1000 [============================>.] - ETA: 0s24s - loss: 5.9018 - acc: 0.6338 - val_loss: 8.0590 - val_acc: 0.5000
Epoch 5/5
 992/1000 [============================>.] - ETA: 0s24s - loss: 5.9018 - acc: 0.6338 - val_loss: 8.0590 - val_acc: 0.5000
